In [1]:
import numpy as np
import pandas as pd
import sklearn

Считываение данных из файла

In [2]:
file = open("mart_export.txt", "r")
data = [line[:-1] for line in file]

Перевод данных в тип set для избавления повторяющихся последовательностей

In [3]:
data_set = set(data)
data_fin = list(data_set)

Обработка данных

In [4]:
i = 0
while True:
    if len(data_fin[i]) < 11:
        data_fin.pop(i)
    elif data_fin[i][0] == '>':
        data_fin.pop(i)
    elif data_fin[i][0] == 'N':
        data_fin.pop(i)
    elif data_fin[i][0] == 'S' and data_fin[i][1] == 'e':
        data_fin.pop(i)
    else:
        i = i+1
    if i == len(data_fin) -1:
        break
data_fin.pop(35578)
len(data_fin)


1164973

Подготовка данных для применения бустинга (первоначальная идея)

In [152]:
data_fina = []
for i in range(len(data_fin)):
    data_fin[i] = data_fin[i].replace('A','1')
    data_fin[i] = data_fin[i].replace('C','2')
    data_fin[i] = data_fin[i].replace('T','3')
    data_fin[i] = data_fin[i].replace('G','4')
    if len(data_fin[i]) == 11 : 
        pos = 6
    else:
        pos = np.random.randint(6, (len(data_fin[i])// 2) + 1)
    data_fina.append(list(data_fin[i] +'0'*(63 - len(data_fin[i]))))
    data_fina[i][-1] = data_fina[i][pos]
    data_fina[i][-2] = str(pos)
    data_fina[i][pos] = '0'
data_final = np.array(data_fina, dtype = int)

In [153]:
data_final.shape

(1164973, 63)

In [154]:
table = pd.DataFrame(data_final)

In [155]:
coll = list(table.columns)
coll[-2] = 'position'
coll[-1] = 'letter_is_drop'
table.columns = coll
table

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,position,letter_is_drop
0,1,4,2,1,1,4,4,2,3,4,...,4,4,4,1,2,3,3,0,24,2
1,1,3,2,3,2,1,4,1,1,2,...,2,3,4,4,4,2,2,0,20,4
2,2,1,1,3,1,3,2,3,3,4,...,1,1,3,3,1,1,1,0,23,3
3,2,3,4,4,1,3,4,2,3,4,...,1,2,1,4,4,4,1,0,14,4
4,4,1,3,4,4,4,4,4,2,3,...,2,2,1,4,2,3,2,0,23,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164968,2,1,2,3,4,3,4,4,4,4,...,2,4,4,4,4,3,4,0,13,1
1164969,3,3,2,2,1,3,1,2,2,2,...,3,3,3,4,2,1,2,0,21,3
1164970,4,3,1,4,2,2,2,0,3,4,...,3,4,1,1,1,3,2,0,7,1
1164971,3,3,4,4,3,4,3,3,4,4,...,3,4,4,3,1,1,4,0,11,2


In [156]:
table.to_csv('date.csv')


Подготовка функции языковой модели)

In [14]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

class LanguageNgramModel:

    def __init__(self, order=1, smoothing=1.0, recursive=0.001):
        self.order = order
        self.smoothing = smoothing
        self.recursive = recursive

    def fit(self, corpus):
        self.counter_ = defaultdict(lambda: Counter())
        self.vocabulary_ = set()
        for i, token in enumerate(corpus[self.order:]):
            context = corpus[i:(i+self.order)]
            self.counter_[context][token] += 1
            self.vocabulary_.add(token)
        self.vocabulary_ = sorted(list(self.vocabulary_))
        if self.recursive > 0 and self.order > 0:
            self.child_ = LanguageNgramModel(self.order-1, self.smoothing, self.recursive)
            self.child_.fit(corpus)

    def get_counts(self, context):
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        freq_dict = self.counter_[local]
        freq = pd.Series(index=self.vocabulary_)
        for i, token in enumerate(self.vocabulary_):
            freq[token] = freq_dict[token] + self.smoothing
        if self.recursive > 0 and self.order > 0:
            child_freq = self.child_.get_counts(context) * self.recursive
            freq += child_freq
        return freq

    def predict_proba(self, context):
        counts = self.get_counts(context)
        return counts / counts.sum()

    def single_log_proba(self, context, continuation):
        result = 0.0
        for token in continuation:
            result += np.log(self.predict_proba(context)[token])
            context += token
        return result

    def single_proba(self, context, continuation):
        return np.exp(self.single_log_proba(context, continuation))

При тестировании обнаружились строки с буквой N. Поэтому дополнительная провека и удаление

In [21]:
data_fin.pop(318451)

'GANCCGGCGCCCTGGTCTGCANTCCCCTGCGGCCTGCTGCTGGATGAGCTCCTGGCGAGC'

In [20]:
for i in range(len(data_fin)):
    for j in range (len(data_fin[i])):
        if data_fin[i][j] == 'N':
            print(i)
            print(data_fin[i])

318451
GANCCGGCGCCCTGGTCTGCANTCCCCTGCGGCCTGCTGCTGGATGAGCTCCTGGCGAGC
318451
GANCCGGCGCCCTGGTCTGCANTCCCCTGCGGCCTGCTGCTGGATGAGCTCCTGGCGAGC


Обучение модели

In [40]:
lang_model = LanguageNgramModel(1)
data_str = ''
lang_model.fit(data_str.join(data_fin[:]))

Тестирование

In [42]:
answer = lang_model.predict_proba('CGTAACGTGAACATCTTTAAATTC')
print(answer)
print('||||||||||||||||||')
if (answer.argmax() == 0):
    print('Следующая буква A с вероятностью: %.3f' %max(answer))
elif (answer.argmax() == 1):
    print('Следующая буква C с вероятностью: %.3f' %max(answer))
elif (answer.argmax() == 2):
    print('Следующая буква T с вероятностью: %.3f' %max(answer))
else:
    print('Следующая буква G с вероятностью: %.3f' %max(answer))

A    0.309232
C    0.296412
G    0.123872
T    0.270483
dtype: float64
||||||||||||||||||
Следующая буква A с вероятностью: 0.309


C:\Users\khahi\AppData\Local\Temp/ipykernel_9172/582917352.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  freq = pd.Series(index=self.vocabulary_)
